# Intro to Plotting

## Objectives

* Plot data from a Pandas DataFrame using Pandas' plotting tools
* Use matplotlib to take more control of figures
* Understand matplotlib's Figures and Axes
* Know some advantages to using seaborn

In [1]:
%matplotlib inline

We'll use the same data from the Pandas lesson:

In [2]:
import pandas as pd

In [3]:
monthly = pd.read_csv('precip_monthly.csv')